<a href="https://colab.research.google.com/github/ffreirea1988/usm/blob/pega/Carga_gastos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Nombre del archivo (se modifica mes a mes)
nmb_input = 'Cuentas TI - 102023 - Octubre 2023.xlsx'

#Mes de cierre
mes = 9

#Último Forecast
nmb_fx = 'FX-23 8+4'


####**Cargar Gastos Reales**

In [3]:
#Cargar pestañas del excel:
df_2701 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112701')
df_2702 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112702')
df_2703 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112703')
df_2704 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112704')
df_2705 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112705')
df_2710 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112710')
df_2711 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112711')
df_2712 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112712')

#Eliminar la última fila de cada tabla (n=1)
n=1
df_2701.drop(df_2701.tail(n).index, inplace = True)
df_2702.drop(df_2702.tail(n).index, inplace = True)
df_2703.drop(df_2703.tail(n).index, inplace = True)
df_2704.drop(df_2704.tail(n).index, inplace = True)
df_2705.drop(df_2705.tail(n).index, inplace = True)
df_2710.drop(df_2710.tail(n).index, inplace = True)
df_2711.drop(df_2711.tail(n).index, inplace = True)
df_2712.drop(df_2712.tail(n).index, inplace = True)

#Concatenar archivos en un solo dataframe con todas las cuentas
#axis=1 unir por filas
#axis=0 unir por columnas
df = pd.concat([df_2701, df_2702, df_2703, df_2704, df_2705, df_2710, df_2711, df_2712], axis=0)

#Reemplazar valor #¡REF!
df['Texto'] = df['Texto'].replace('#¡REF!','REF_')

#Eliminar mes no cerrado:
df=df[df['Período contable'] <= mes]
df_mes_abierto = df[df['Período contable'] > mes]

#Agregar columna Tipo = Real 2023
df['Tipo'] = 'Real-2023'

####**Auxiliares para Clasificar Gastos Reales**

In [4]:
#Cargar auxiliar de clasificación para Gastos Reales
nmb_aux = 'Aux_Clasificación.xlsx'
df_aux_oc = pd.read_excel(f'/content/{nmb_aux}', sheet_name='Aux_Clasificación_OC')
df_aux_txt = pd.read_excel(f'/content/{nmb_aux}', sheet_name='Aux_Clasificación_Txt')

#1ra Agrupación
#Agrupar dataframe con clasificación por OC:
df_clas = df.merge(df_aux_oc, how='left', on='Documento compras')

#Eliminar columnas duplicadas por el join:
df_clas.drop(['Cuenta_y'], axis=1, inplace=True)

#Renombrar columnas modificadas por el join ("_x"):
df_clas = df_clas.rename(columns={'Cuenta_x':'Cuenta'})

#2da Agrupación
#Textos en minúsculas para hacer join:
df_aux_txt['Texto cab.documento_minusc']=df_aux_txt['Texto cab.documento'].str.lower()
df_clas['Texto cab.documento_minusc']=df_clas['Texto cab.documento'].str.lower()
df_aux_txt['Texto_minusc']=df_aux_txt['Texto'].str.lower()
df_clas['Texto_minusc']=df_clas['Texto'].str.lower()

#Agrupar dataframe con clasificación por Texto:
df_clas_2 = df_clas.merge(df_aux_txt, how='left', on=['Texto_minusc', 'Cuenta', 'Nº documento', 'Texto cab.documento_minusc'])

#Agrupar clasificaciones:
df_clas_2['Clasificación_N1']=df_clas_2['Clasificación_N1_x'].fillna('')+df_clas_2['Clasificación_N1_y'].fillna('')
df_clas_2['Clasificación_N2']=df_clas_2['Clasificación_N2_x'].fillna('')+df_clas_2['Clasificación_N2_y'].fillna('')

#Eliminar columnas duplicadas por el join:
df_clas_2.drop(['KEY','Clasificación_N1_x', 'Clasificación_N2_x',
                'Clasificación_N1_y', 'Clasificación_N2_y',
                'Texto cab.documento_minusc', 'Texto_minusc',
                'Texto cab.documento_y', 'Texto_y',
                'Nmb_Cuenta'
                ], axis=1, inplace=True)

#Renombrar columnas modificadas por el join ("_x"):
df_clas_2 = df_clas_2.rename(columns={'Texto cab.documento_x':'Texto cab.documento',
                                   'Texto_x':'Texto'})

#Definir Dataframe Final con los ajustes implementados:
df_1 = df_clas_2

####**Agrupar Gastos Reales/Presupuesto/Forecast**

In [5]:
#Cargar Presupuesto y Forecast:
#Cargar POA-2023
nmb_poa_fx = 'Base POA-FX-Clasificación-Ceco-Cta.xlsx'
df_poa_fx = pd.read_excel(f'/content/{nmb_poa_fx}', sheet_name='TH_POA_FX')

#Concatenar dataframes:
df_concat = pd.concat([df_1, df_poa_fx],ignore_index=True)
df_concat['Periodo']=df_concat['Período contable'].fillna(0)+df_concat['Mes'].fillna(0)
df_concat['Monto_CLP']=df_concat['Importe en moneda local'].fillna(0)+df_concat['Valor'].fillna(0)
df_concat['CeCo']=df_concat['Centro de coste'].fillna(0)+df_concat['CC'].fillna(0)

#Eliminar columnas duplicadas al concatenar:
df_concat.drop(['Importe en moneda local','Valor',
                'Período contable', 'Mes',
                'Centro de coste', 'CC',
                'Nombre Cuenta'
                ], axis=1, inplace=True)

#Definir Dataframe Final con los ajustes implementados:
df_2 = df_concat

####**Auxiliares para agregar Ceco-Cuenta**

In [6]:
#Cargar auxiliar de clasificación para Gastos Reales
nmb_aux = 'Aux_Clasificación.xlsx'
df_aux_ceco = pd.read_excel(f'/content/{nmb_aux}', sheet_name='Aux_Cecos')

df = df_2

#Agregar nombre de cuentas:
df['Nmb_Cuenta'] = df['Cuenta'].replace({
        31112701:'Leasing Equipos',
        31112702:'Evolutivos y Correctivos',
        31112703:'Redes',
        31112704:'Data Center',
        31112705:'Licencias De Software',
        31112710:'Soporte Operacional',
        31112711:'Evolutivos Mayores',
        31112712:'Soporte de Sistemas'
        })

#Agregar datos de los Cecos al dataframe:
df_ceco = df.merge(df_aux_ceco, how='left', left_on='CeCo', right_on='CeCo')

#Eliminar columnas duplicadas por el join:
#df_ceco.drop(['CeCo'], axis=1, inplace=True)

#Agregar Ceco_TI
df_ceco['Ceco_TI'] = ['GTI' if s=='GTI' else 'Otro' for s in df_ceco['Subgerencia']]

#Definir Dataframe Final con los ajustes implementados:
df_3 = df_ceco

In [ ]:
df_3.columns

Index(['Texto cab.documento', 'Texto', 'Cuenta', 'Moneda local', 'Referencia',
       'Material', 'Nombre del usuario', 'Nº documento', 'Fecha de documento',
       'Fe.contabilización', 'Importe en ML3', 'Clase de documento', 'Orden',
       'Sociedad', 'Documento compras', 'Grafo', 'Elemento PEP',
       'Importe libro mayor', 'Moneda libro mayor', 'Tp.cambio efectivo',
       'Posición', 'Ejercicio / mes', 'Clave contabiliz.', 'Tipo',
       'Clasificación_N1', 'Clasificación_N2', 'Periodo', 'Monto_CLP', 'CeCo',
       'Nmb_Cuenta', 'Gerencia', 'Subgerencia', 'Nombre CeCo', 'GrupoCeCo',
       'Area', 'Dependencia', 'Ceco_TI'],
      dtype='object')

#### **Definir Last Forecast**

In [7]:
#Crear Last Forecast:
last_fx_real = df_3[df_3['Tipo']=='Real-2023']
last_fx_estimado = df_3[(df_3['Tipo']==nmb_fx)&(df_3['Periodo']>mes)]

last_fx = pd.concat([last_fx_real, last_fx_estimado], axis=0)
last_fx['Tipo']='LF-2023'

#Concatenar:
df_4 = pd.concat([df_3, last_fx], axis=0)

In [8]:
print(last_fx_real.shape)
print(last_fx_estimado.shape)
print(last_fx.shape)
print('Real: ',last_fx_real.shape[0])
print('Estimado: ',last_fx_estimado.shape[0])
print('Real + Estimado: ',last_fx_real.shape[0]+last_fx_estimado.shape[0])
print('Last Forecast: ',last_fx.shape[0])


(25607, 37)
(3560, 37)
(29167, 37)
Real:  25607
Estimado:  3560
Real + Estimado:  29167
Last Forecast:  29167


####**Exportar archivo**#

In [9]:
#Definir dataframe final:
df_final = df_4

#Incorporar Tipo_Mes:
df_final['Tipo_Mes'] = df_final['Tipo'] + '_' + df_final['Periodo'].astype(int).astype(str)

In [12]:
df_final['Tipo'].unique()

array(['Real-2023', 'POA-2023', 'FX-23 6+6', 'FX-23 8+4', 'POA-2024',
       'LF-2023'], dtype=object)

In [16]:
#Eliminar FX anteriores y "Real-2023":
df_final = df_final[df_final['Tipo'] != 'Real-2023']
df_final = df_final[df_final['Tipo'] != 'FX-23 6+6']

In [10]:
#Reordenar columnas
df_output = df_final[['Tipo_Mes', 'Tipo', 'Periodo',
               'CeCo', 'Ceco_TI', 'Nombre CeCo',
               'Gerencia', 'Subgerencia', 'GrupoCeCo', 'Area', 'Dependencia',
               'Cuenta', 'Nmb_Cuenta',
               'Clasificación_N1', 'Clasificación_N2',
               'Monto_CLP',
               'Texto cab.documento', 'Texto', 'Moneda local', 'Referencia',
               'Material', 'Nombre del usuario', 'Nº documento', 'Fecha de documento',
               'Fe.contabilización', 'Importe en ML3', 'Clase de documento', 'Orden',
               'Sociedad', 'Documento compras', 'Grafo', 'Elemento PEP',
               'Importe libro mayor', 'Moneda libro mayor', 'Tp.cambio efectivo',
               'Posición', 'Ejercicio / mes', 'Clave contabiliz.'
               ]]

#Nombre del archivo de salida:
nmb_output = nmb_input.replace('Cuentas TI','Gastos TI')

#Extraer archivo a excel
df_output.to_excel(nmb_output, index=False)

#**Ayuda**

In [ ]:
#Buscar un gasto:
df_final.loc[df_final['Nº documento']==119353799].head(1)
#df_aux_txt.loc[df_final['Nº documento']==119353799].head(1)

,Período contable,Texto cab.documento_x,Texto_x,Cuenta,Importe en moneda local,Moneda local,Centro de coste,Referencia,Material,Nombre del usuario,...,Moneda libro mayor,Tp.cambio efectivo,Posición,Ejercicio / mes,Clave contabiliz.,Nmb_Cuenta,Texto cab.documento_y,Texto_y,Clasificación_N1,Clasificación_N2
42,8.0,GTI,Provision Leasing Notebook,31112701.0,52691,CLP,11000.0,JOAQUIN DANUS,NaN,RROJAS,...,CLP,"1,00000",3.0,2023/08,40.0,Leasing Equipos,NaN,NaN,,


In [ ]:
#Agrupar datos
df_2705['Importe en moneda local'].groupby(by=df_2705['Período contable']).count()

Período contable
1.0     1083
2.0     1115
3.0     1597
4.0      877
5.0     1258
6.0      980
7.0      973
8.0      917
9.0      729
10.0      34
Name: Importe en moneda local, dtype: int64

In [ ]:
df_clas_2['Cuenta'].unique()

array([31112701., 31112702., 31112703., 31112704., 31112705., 31112710.,
       31112711., 31112712.])

In [ ]:
df_clas_2['Período contable'].unique()

array([1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [ ]:
df_clas_3.columns

Index(['Período contable', 'Texto cab.documento', 'Texto', 'Cuenta',
       'Importe en moneda local', 'Moneda local', 'Centro de coste',
       'Referencia', 'Material', 'Nombre del usuario', 'Nº documento',
       'Fecha de documento', 'Fe.contabilización', 'Importe en ML3',
       'Clase de documento', 'Orden', 'Sociedad', 'Documento compras', 'Grafo',
       'Elemento PEP', 'Importe libro mayor', 'Moneda libro mayor',
       'Tp.cambio efectivo', 'Posición', 'Ejercicio / mes',
       'Clave contabiliz.', 'Nmb_Cuenta', 'Clasificación_N1',
       'Clasificación_N2', 'Gerencia', 'Subgerencia', 'Nombre CeCo',
       'GrupoCeCo', 'Area', 'Dependencia'],
      dtype='object')